# HACKATHON_Geonames

### **Заказчик**

- Карьерный центр Яндекс Практикум

### **Описание проекта**

**Цель:**
:
- Сопоставление произвольных гео названий с унифицированными именами geonames для внутреннего использования Карьерным центр

**Задачи:**

- Создать решение для подбора наиболее подходящих названий с geonames. Например Ереван -> Yerevan

- На примере РФ и стран наиболее популярных для релокации - Беларусь, Армения, Казахстан, Кыргызстан, Турция, Сербия. Города с населением от 15000 человек (с возможностью масштабирования на сервере заказчика)


- Возвращаемые поля geonameid, name, region, country, cosine similarity
- формат данных на выходе: список словарей, например [{dict_1}, {dict_2}, …. {dict_n}] где словарь - одна запись с указанными полями



**Задачи опционально:**


- возможность настройки количества выдачи подходящих названий (например в параметрах метода)


- коррекция ошибок и опечаток. Например Моченгорск -> Monchegorsk


- хранение в PostgreSQL данных geonames


- хранение векторизованных промежуточных данных в PostgreSQL


- предусмотреть методы для настройки подключения к БД


- предусмотреть метод для инициализации класса (первичная векторизация geonames)


- предусмотреть методы для добавления векторов новых гео названий


**Результат:**


- тетрадка с решением задачи (описание проекта, исследование, методы решения)
- python-скрипт, содержащий функцию (класс), для интеграции в систему Заказчиканкцию (класс), для интеграции в систему Заказчикаями


### **Описание данных**
Используемые таблицы с geonames:

- admin1CodesASCII
- alternateNamesV2
- cities15000
- countryInfo
- при необходимости любые другие открытые данные
- таблицы geonames можно скачать здесь http://download.geonames.org/export/dump/


### Загрузка данных

In [ ]:
import pandas as pd
import numpy as np
import random

import sys
import re
import os

import torch
import logging

from sqlalchemy import create_engine,text
from sqlalchemy.engine.url import URL
from sentence_transformers import (
    SentenceTransformer,
    util,
    models,
    InputExample,
    losses,
    evaluation
)
from torch.utils.data import DataLoader
import psycopg2
from psycopg2 import Error
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 123

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [ ]:
RELOC_COUNTRIES = ['RU', 'BY', 'KG', 'KZ', 'AM', 'GE', 'RS', 'ME']

In [ ]:
DATABASE = {
    'drivername': 'postgresql',
    'username': 'postgres',
    'password': '2017',
    'host': 'localhost',
    'port': 5432,
    'database': 'postgres',
    'query': {}
}

engine = create_engine(URL(**DATABASE))

In [ ]:
DATA = Path('DATABASE')
MODEL = Path('_'.join(RELOC_COUNTRIES))

## Загрузка датасетов

In [ ]:
admin_codes = pd.read_csv(
    DATA/'admin1CodesASCII.txt',
    delimiter ='\t',
    header=None,
    names=[
        'code',
        'name',
        'region',
        'geonameid',
    ]
)

admin_codes.head()

,code,name,region,geonameid
0,AD.06,Sant Julià de Loria,Sant Julia de Loria,3039162
1,AD.05,Ordino,Ordino,3039676
2,AD.04,La Massana,La Massana,3040131
3,AD.03,Encamp,Encamp,3040684
4,AD.02,Canillo,Canillo,3041203


In [ ]:
admin_codes.to_sql('admin_codes', con=engine,  if_exists='replace')

881

In [ ]:
query = 'SELECT * FROM admin_codes LIMIT 5'
pd.read_sql_query(query, con=engine)

,index,code,name,region,geonameid
0,0,AD.06,Sant Julià de Loria,Sant Julia de Loria,3039162
1,1,AD.05,Ordino,Ordino,3039676
2,2,AD.04,La Massana,La Massana,3040131
3,3,AD.03,Encamp,Encamp,3040684
4,4,AD.02,Canillo,Canillo,3041203


In [ ]:
countries = pd.read_csv(
    DATA/'countryInfo.txt',
    delimiter ='\t',
    header=None,
    names=[
        'country_code',
        'ISO_3',
        'ISO_numeric',
        'fips',
        'country',
        'capital',
        'area',
        'population',
        'continent',
        'tld',
        'currency_code',
        'currency_name',
        'phone',
        'postal_code_format',
        'postal_code_regex',
        'languages',
        'geonameid',
        'neighbours',
        'equivalent_fips_code'
    ],
    usecols = [
        'geonameid',
        'country_code',
        'capital',
        'country'
    ]
).dropna()
countries.head()

,country_code,country,capital,geonameid
49,#ISO,Country,Capital,geonameid
50,AD,Andorra,Andorra la Vella,3041565
51,AE,United Arab Emirates,Abu Dhabi,290557
52,AF,Afghanistan,Kabul,1149361
53,AG,Antigua and Barbuda,St. John's,3576396


In [ ]:
countries.to_sql('countries', con=engine,  if_exists='replace')

246

In [ ]:
query =  'SELECT * FROM countries'
pd.read_sql_query(query, con=engine)

,index,country_code,country,capital,geonameid
0,49,#ISO,Country,Capital,geonameid
1,50,AD,Andorra,Andorra la Vella,3041565
2,51,AE,United Arab Emirates,Abu Dhabi,290557
3,52,AF,Afghanistan,Kabul,1149361
4,53,AG,Antigua and Barbuda,St. John's,3576396
...,...,...,...,...,...
241,297,ZA,South Africa,Pretoria,953987
242,298,ZM,Zambia,Lusaka,895949
243,299,ZW,Zimbabwe,Harare,878675
244,300,CS,Serbia and Montenegro,Belgrade,8505033


In [ ]:
alternate = pd.read_csv(
    DATA/'alternateNamesV2.txt',
    delimiter ='\t',
    header=None,
    low_memory= False,
    names=[
        'alternate_name_id',
        'geonameid',
        'alternate_lang',
        'alternate_name',
        'is_preferred_name',
        'is_short_name',
        'is_colloquial',
        'is_historic',
        'use_from',
        'Use_to'
    ],
    usecols = [
        'alternate_name_id',
        'geonameid',
        'alternate_lang',
        'alternate_name',
    ]
)

In [ ]:
alternate.to_sql('alternate', con=engine, if_exists='replace')

367

In [ ]:
query = 'SELECT * FROM alternate LIMIT 5'
pd.read_sql_query(query, con=engine)

,index,alternate_name_id,geonameid,alternate_lang,alternate_name
0,0,1284819,2994701,None,Roc Mélé
1,1,1284820,2994701,None,Roc Meler
2,2,4285256,3007683,None,Pic des Langounelles
3,3,1291197,3017832,None,Pic de les Abelletes
4,4,4290387,3017832,None,Pic de la Font-Nègre


In [ ]:
cities15000 = pd.read_csv(
    DATA/'cities15000.txt',
    delimiter ='\t',
    header=None,
    low_memory= False,
    names=[
        'geonameid',
        'name',
        'region',
        'alternate_name',
        'latitude',
        'longitude',
        'feature_class',
        'feature_code',
        'country_code',
        'cc2',
        'admin1_code',
        'admin2_code',
        'admin3_code',
        'admin4_code',
        'population',
        'elevation',
        'dem',
        'timezone',
        'modification_date'
    ],
    usecols = [
        'geonameid',
        'name',
        'region',
        'alternate_name',
        'country_code',
        'admin1_code',
   ]).dropna()

cities15000.head()

,geonameid,name,region,alternate_name,country_code,admin1_code
0,3040051,les Escaldes,les Escaldes,"Ehskal'des-Ehndzhordani,Escaldes,Escaldes-Engo...",AD,08
1,3041563,Andorra la Vella,Andorra la Vella,"ALV,Ando-la-Vyey,Andora,Andora la Vela,Andora ...",AD,07
2,290594,Umm Al Quwain City,Umm Al Quwain City,"Oumm al Qaiwain,Oumm al Qaïwaïn,Um al Kawain,U...",AE,07
3,291074,Ras Al Khaimah City,Ras Al Khaimah City,"Julfa,Khaimah,RAK City,RKT,Ra's al Khaymah,Ra'...",AE,05
4,291580,Zayed City,Zayed City,"Bid' Zayed,Bid’ Zayed,Madinat Za'id,Madinat Za...",AE,01


In [ ]:
cities15000.to_sql('cities15000', con=engine,  if_exists='replace')

817

In [ ]:
query = 'SELECT * FROM cities15000 LIMIT 5'
pd.read_sql_query(query, con=engine)

,index,geonameid,name,region,alternate_name,country_code,admin1_code
0,0,3040051,les Escaldes,les Escaldes,"Ehskal'des-Ehndzhordani,Escaldes,Escaldes-Engo...",AD,08
1,1,3041563,Andorra la Vella,Andorra la Vella,"ALV,Ando-la-Vyey,Andora,Andora la Vela,Andora ...",AD,07
2,2,290594,Umm Al Quwain City,Umm Al Quwain City,"Oumm al Qaiwain,Oumm al Qaïwaïn,Um al Kawain,U...",AE,07
3,3,291074,Ras Al Khaimah City,Ras Al Khaimah City,"Julfa,Khaimah,RAK City,RKT,Ra's al Khaymah,Ra'...",AE,05
4,4,291580,Zayed City,Zayed City,"Bid' Zayed,Bid’ Zayed,Madinat Za'id,Madinat Za...",AE,01


In [ ]:
data = pd.concat([admin_codes,countries,cities15000,alternate], ignore_index = True)
data.head()

,code,name,region,geonameid,country_code,country,capital,alternate_name,admin1_code,alternate_name_id,alternate_lang
0,AD.06,Sant Julià de Loria,Sant Julia de Loria,3039162,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AD.05,Ordino,Ordino,3039676,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AD.04,La Massana,La Massana,3040131,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AD.03,Encamp,Encamp,3040684,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AD.02,Canillo,Canillo,3041203,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.shape

(16064311, 11)

In [ ]:
data['code'] = data.country_code + '.' +  data.admin1_code
#data = data.drop('admin1_code', axis = `1)
data = data[data.country_code.isin(RELOC_COUNTRIES)]
data.head()

,code,name,region,geonameid,country_code,country,capital,alternate_name,admin1_code,alternate_name_id,alternate_lang
3888,NaN,NaN,NaN,174982,AM,Armenia,Yerevan,NaN,NaN,NaN,NaN
3914,NaN,NaN,NaN,630336,BY,Belarus,Minsk,NaN,NaN,NaN,NaN
3957,NaN,NaN,NaN,614540,GE,Georgia,Tbilisi,NaN,NaN,NaN,NaN
3993,NaN,NaN,NaN,1527747,KG,Kyrgyzstan,Bishkek,NaN,NaN,NaN,NaN
4003,NaN,NaN,NaN,1522867,KZ,Kazakhstan,Nur-Sultan,NaN,NaN,NaN,NaN


In [ ]:
data.shape

(1353, 11)

In [ ]:
data = data[['code','geonameid', 'name', 'alternate_name', 'region','country','country_code']]\
                      .drop_duplicates()\
                      .reset_index(drop=True)

data.head()

,code,geonameid,name,alternate_name,region,country,country_code
0,NaN,174982,NaN,NaN,NaN,Armenia,AM
1,NaN,630336,NaN,NaN,NaN,Belarus,BY
2,NaN,614540,NaN,NaN,NaN,Georgia,GE
3,NaN,1527747,NaN,NaN,NaN,Kyrgyzstan,KG
4,NaN,1522867,NaN,NaN,NaN,Kazakhstan,KZ


In [ ]:
data.shape

(1353, 7)

In [ ]:
data.alternate_name = data.alternate_name.str.split(',')
data = data.explode('alternate_name')
data = data[data.name!=data.alternate_name]
data = data.drop_duplicates(subset= ['name', 'alternate_name'])

In [ ]:
data

,code,geonameid,name,alternate_name,region,country,country_code
0,NaN,174982,NaN,NaN,NaN,Armenia,AM
8,AM.08,174875,Kapan,Ghap'an,Kapan,NaN,AM
8,AM.08,174875,Kapan,Ghapan,Kapan,NaN,AM
8,AM.08,174875,Kapan,Ghap’an,Kapan,NaN,AM
8,AM.08,174875,Kapan,Kafan,Kapan,NaN,AM
...,...,...,...,...,...,...,...
1350,RU.47,8521440,Dzerzhinsky,Дзержинский,Dzerzhinsky,NaN,RU
1351,RU.32,11886891,Fedorovskiy,Fedorovskij,Fedorovskiy,NaN,RU
1351,RU.32,11886891,Fedorovskiy,Федоровский,Fedorovskiy,NaN,RU
1352,RU.08,12041452,Mezgor'e,Mezhgor'e,Mezgor'e,NaN,RU


In [ ]:
data.shape

(19016, 7)

In [ ]:
data['example'] = (data[['name', 'alternate_name']]
                        .apply(lambda x:InputExample(texts=list(x)),
                               axis = 1))

train_examples = data['example'].to_list()

In [ ]:
labse = SentenceTransformer('sentence-transformers/LaBSE')

In [ ]:
names = data.name.drop_duplicates().values
names[:10]

array([nan, 'Kapan', 'Goris', 'Hats’avan', 'Artashat', 'Ararat',
       'Yerevan', 'Vagharshapat', 'Stepanavan', 'Spitak'], dtype=object)

In [ ]:
embeddings = labse.encode(names)
embeddings.shape

(1325, 768)

In [ ]:
def get_sim(geoname, names=names, embeddings= embeddings,model =labse, top_k=3):
    result= pd.DataFrame(util.semantic_search(model.encode(geoname),embeddings, top_k= top_k) [0])
    return result.assign(name=names[result.corpus_id])

In [ ]:
get_sim('Масква').drop('corpus_id', axis=1).to_dict(orient='records')

[{'score': 0.9397482872009277, 'name': 'Moscow'},
 {'score': 0.7571744918823242, 'name': 'Moskovskiy'},
 {'score': 0.6858565807342529, 'name': 'Minsk'}]

In [ ]:
get_sim('сантк- питербурк').drop('corpus_id', axis=1).to_dict(orient='records')

[{'score': 0.5200719237327576, 'name': 'Saint Petersburg'},
 {'score': 0.43851250410079956, 'name': 'Pinsk'},
 {'score': 0.41483384370803833, 'name': 'Sestroretsk'}]

In [ ]:
get_sim('Краснодар').drop('corpus_id', axis=1).to_dict(orient='records')

[{'score': 0.9180408120155334, 'name': 'Krasnodar'},
 {'score': 0.692857563495636, 'name': 'Krasnoyarsk'},
 {'score': 0.6585469245910645, 'name': 'Kazan'}]

In [ ]:
get_sim('Moscow').drop('corpus_id', axis=1).to_dict(orient='records')

[{'score': 1.000000238418579, 'name': 'Moscow'},
 {'score': 0.8265248537063599, 'name': 'Moskovskiy'},
 {'score': 0.7368156909942627, 'name': 'Mostovskoy'}]

In [ ]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.MultipleNegativesRankingLoss(model=labse)

In [ ]:
labse.fit(train_objectives=[(train_dataloader, train_loss)], epochs=3)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1189 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1189 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1189 [00:00<?, ?it/s]

In [ ]:
labse.save('labse-fine-tune_data_RU_BY_KG_KZ_AM_GE_RS_ME')

In [ ]:
%load_ext autoreload
%autoreload 2
from test_module import get_similar
